# ANN to SNN conversion

In this notebook we try to import an ANN ("analog neural network", a regular non-spiking network) and convert it to a SNN (spiking neural network). 

For this we have generated some an ANN using [this](https://github.com/spikingevolution/evolution-strategies/blob/master/evolution-strategies.ipynb) jupyter notebook. We picked a snapshot of it and saved it as `snapshot_00423.h5`. We use this network as our input. We will convert this network into a an equivalent SNN. And then we will execute both networks and see how they compare. 

## roadmap / todo

- [ ] first we rebuild this docker-container and install SNN-toolbox
- [ ] then we copy the snnToNest example from [the official snn_toolbox repository](https://github.com/NeuromorphicProcessorProject/snn_toolbox/blob/master/examples/mnist_keras_nest.py). We use that script as a base
- [ ] generate a better ANN model. (The current one just stands still and does nothing)
- [ ] import the snapshot and convert it to a SNN
- [ ] run this SNN using PyNN and dummy-data.
- [ ] install the runtime-environment which we need to run the original ANN
- [ ] run the original ANN using the same dummy-data
- [ ] compare both
- [ ] install the physics environment that was used to train the ANN
- [ ] run the original ANN using the in that environment
- [ ] run the SNN in that environment

## problems

* none so far

## potential problems

- snn_toolbox has lots of parameters and optimisations. Creating an SNN that actually works may be hard
  - solution: read paper
- creating run both ANN and SNN concurrently requires dependancies for both. It may be a pain to set up both, and to create equivalent input for both
  - solution: it probably takes some time
- ?

In [1]:
"""imports for SNN Toolbox.
"""

import os
import time
import numpy as np

from tensorflow import keras 
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, Dropout
from tensorflow.keras.datasets import mnist
# from keras.utils import np_utils
from tensorflow.python.keras import utils as np_utils

from snntoolbox.bin.run import main
from snntoolbox.utils.utils import import_configparser


Using TensorFlow backend.


In [2]:
# WORKING DIRECTORY #
#####################

# Define path where model and output files will be stored.
# The user is responsible for cleaning up this temporary directory.
path_wd = os.path.abspath(os.path.join(os.path.dirname(os.path.realpath(
    "snntest")), '..', 'temp', str(time.time())))
os.makedirs(path_wd)

In [3]:
from shutil import copyfile
ann_path='snapshot_00423.h5'
ann_name="example_ann"
target_path = os.path.join(path_wd,ann_name)
copyfile(ann_path, target_path+".h5")
copyfile('x_test.npz', os.path.join(path_wd,'x_test.npz'))
copyfile('y_test.npz', os.path.join(path_wd,'y_test.npz'))

# for the config below
model_name = ann_name

In [4]:
# SNN TOOLBOX CONFIGURATION #
#############################

# Create a config file with experimental setup for SNN Toolbox.
configparser = import_configparser()
config = configparser.ConfigParser()

config['paths'] = {
    'path_wd': path_wd,             # Path to model.
    'dataset_path': path_wd,        # Path to dataset.
    'filename_ann': model_name      # Name of input model.
}

config['tools'] = {
    'evaluate_ann': True,           # Test ANN on dataset before conversion.
    'normalize': True,              # Normalize weights for full dynamic range.
     'normalize': False,    
}

config['simulation'] = {
    'simulator': 'nest',            # Chooses execution backend of SNN toolbox.
    'duration': 50,                 # Number of time steps to run each sample.
    'num_to_test': 5,               # How many test samples to run.
    'batch_size': 1,                # Batch size for simulation.
}

config['output'] = {
    'plot_vars': {                  # Various plots (slows down simulation).
        'spiketrains',              # Leave section empty to turn off plots.
        'spikerates',
        'activations',
        'correlation',
        'v_mem',
        'error_t'
    }
}

# Store config file.
config_filepath = os.path.join(path_wd, 'config')
with open(config_filepath, 'w') as configfile:
    config.write(configfile)
    
# RUN SNN TOOLBOX #
###################

main(config_filepath)

Initializing nest simulator...



/opt/conda/lib/python3.7/site-packages/pyNN/nest/__init__.py:55: UserWarning: Unable to install NEST extensions. Certain models may not be available.
Further details: DynamicModuleManagementError in Install: Module 'pynn_extensions' could not be opened.
The dynamic loader returned the following error: 'file not found'.

Please check LD_LIBRARY_PATH (OSX: DYLD_LIBRARY_PATH)!
  warnings.warn("Unable to install NEST extensions. Certain models may not be available.\nFurther details: {}".format(err))


Loading data set from '.npz' files in /home/jovyan/base_repository/temp/1573654003.820306.



ValueError: Unknown layer: ObservationNormalizationLayer

In [ ]:
!ls /home/jovyan/base_repository/temp/1573645371.7412007/